In [67]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import h5py
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import os
from torchvision import transforms
import sys
from model import CNNSAM
import numpy as np

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# cleaning
df = pd.read_csv('/mnt/c/Users/kmitchell/Documents/GitHub/Thesis/temp_data/hsi_tensors/master.csv')
df = df.drop(['Unnamed: 0.1'], axis=1)
df = df.drop_duplicates()
df = df.drop(index=36)
df.dropna(subset=['taxonID_x'], inplace=True)
# df = df[df['taxonID_x'] == 'LITU']
df['distance'] = df['distance'].astype(float)
# df = df.loc[df.groupby('geometry')['distance'].idxmin()]
df = df[df['distance'] < 10]

In [68]:
df['phenophaseName'].value_counts()

Leaves                  61
Open flowers            41
Increasing leaf size    33
Breaking leaf buds      30
Initial growth          16
Colored leaves          13
Falling leaves           9
Name: phenophaseName, dtype: int64

In [47]:
phenogeo = pd.read_csv('/mnt/c/Users/kmitchell/Documents/GitHub/Thesis/temp_data/phenogeo.csv')

/tmp/ipykernel_47561/3570552355.py:1: DtypeWarning: Columns (0,1,2,3,4,7,11,13,21,22,24,25,27,28,29,31,32,33,34,35,37,38,39,40,41,42,43,45,46,47,48,49,50,51,52,53,54,57,58,59,60,61,64,68,70,78,79,80,81,82,83,84,85,86,88,89,90,91,93,94,99,107,111,112,113,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  phenogeo = pd.read_csv('/mnt/c/Users/kmitchell/Documents/GitHub/Thesis/temp_data/phenogeo.csv')


In [48]:
AOP_flyovers = {'SCBI': ['2016-07', '2017-07', '2019-06', '2021-08', '2022-05', '2023-06'], 'ORNL': ['2015-08', '2016-06', '2017-09'], 'GRSM': ['2015-08', '2016-06', '2017-10', '2018-05', '2021-06', '2022-05', '2022-09'], 'SERC': ['2016-07', '2017-07', '2017-08', '2019-05', '2021-08', '2022-05']}

for row in df.iterrows():
    individual_id = row[1]['individualID']
    image_path = row[1]['image_path']
    year = image_path.split('_')[0]
    site = image_path.split('_')[1]
    date = [x for x in AOP_flyovers[site] if year in x][0]
    date = pd.to_datetime(date)


    site_df = phenogeo[phenogeo['siteID'] == site]
    individual_df = site_df[site_df['individualID'] == individual_id]
    individual_df['date_y'] = pd.to_datetime(individual_df['date_y'])

    individual_df['date_diff'] = (individual_df['date_y'] - date).abs()
    closest_observation = individual_df.loc[individual_df['date_diff'].idxmin()]

    if closest_observation['phenophaseName'] == 'Open flowers':
        df.at[row[0], 'flowering'] = 0
    else:
        flowering_individual_df = individual_df[individual_df['phenophaseName'] == 'Open flowers']

        flowering_individual_df.loc[:, 'date_diff'] = (flowering_individual_df['date_y'] - date).abs()

        closest_flowering_observation = flowering_individual_df.loc[flowering_individual_df['date_diff'].idxmin()]

        if closest_flowering_observation['date_y'] < date:
            df.at[row[0], 'flowering'] = 1
        else:
            df.at[row[0], 'flowering'] = -1

/tmp/ipykernel_47561/572866936.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_df['date_y'] = pd.to_datetime(individual_df['date_y'])
/tmp/ipykernel_47561/572866936.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_df['date_diff'] = (individual_df['date_y'] - date).abs()
/tmp/ipykernel_47561/572866936.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the 

In [49]:
df['flowering'].value_counts()

 1.0    34
-1.0    11
 0.0     7
Name: flowering, dtype: int64

In [50]:
import os
import requests
import json
import itertools
from itertools import chain
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import math

SERVER = 'http://data.neonscience.org/api/v0/'

# Read the config.yml file
with open('../config.yml', 'r') as file:
    config = yaml.safe_load(file)

# Get the sites from the config
sites = config['sites']

# Set the data path
input_path = '../input_data/phenology'
output_path = '../temp_data'
phenogeo_path = os.path.join(output_path, 'phenogeo.csv')

phenogeo = pd.read_csv(phenogeo_path)

site_months = {}

for site in sites:
    SITECODE = site

    #Define the url, using the sites/ endpoint
    url = SERVER+'sites/'+SITECODE

    #Request the url
    site_request = requests.get(url)

    #Convert the request to Python JSON object
    site_json = site_request.json()

    #Set the Ecosystem structure (CHM) data product
    # PRODUCTCODE = 'DP1.30001.001' # LiDAR slant range waveform
    # PRODUCTCODE = 'DP1.30003.001' # Discrete return LiDAR point cloud
    # PRODUCTCODE = 'DP3.30010.001' # High-resolution orthorectified camera imagery mosiac
    PRODUCTCODE = 'DP3.30010.001' # High-resolution orthorectified camera imagery



    #Get available months of Ecosystem structure data products for TEAK site
    #Loop through the 'dataProducts' list items (each one is a dictionary) at the site
    for product in site_json['data']['dataProducts']: 
        #if a list item's 'dataProductCode' dict element equals the product code string
        if(product['dataProductCode'] == PRODUCTCODE): 
            #print the available months
            print(site)
            print('Available Months: ',product['availableMonths'])
            months = [pd.to_datetime(month).strftime('%Y-%m') for month in product['availableMonths']]
            site_months[site] = months
            print('URLs for each Month:')
            #print the available URLs
            for url in product['availableDataUrls']:
                print(url)

/tmp/ipykernel_47561/1722429160.py:26: DtypeWarning: Columns (0,1,2,3,4,7,11,13,21,22,24,25,27,28,29,31,32,33,34,35,37,38,39,40,41,42,43,45,46,47,48,49,50,51,52,53,54,57,58,59,60,61,64,68,70,78,79,80,81,82,83,84,85,86,88,89,90,91,93,94,99,107,111,112,113,116,117) have mixed types. Specify dtype option on import or set low_memory=False.
  phenogeo = pd.read_csv(phenogeo_path)


SCBI
Available Months:  ['2016-07', '2017-07', '2019-06', '2021-08', '2022-05', '2023-06']
URLs for each Month:
https://data.neonscience.org/api/v0/data/DP3.30010.001/SCBI/2016-07
https://data.neonscience.org/api/v0/data/DP3.30010.001/SCBI/2017-07
https://data.neonscience.org/api/v0/data/DP3.30010.001/SCBI/2019-06
https://data.neonscience.org/api/v0/data/DP3.30010.001/SCBI/2021-08
https://data.neonscience.org/api/v0/data/DP3.30010.001/SCBI/2022-05
https://data.neonscience.org/api/v0/data/DP3.30010.001/SCBI/2023-06
ORNL
Available Months:  ['2015-08', '2016-06', '2017-09']
URLs for each Month:
https://data.neonscience.org/api/v0/data/DP3.30010.001/ORNL/2015-08
https://data.neonscience.org/api/v0/data/DP3.30010.001/ORNL/2016-06
https://data.neonscience.org/api/v0/data/DP3.30010.001/ORNL/2017-09
GRSM
Available Months:  ['2015-08', '2016-06', '2017-10', '2018-05', '2021-06', '2022-05', '2022-09']
URLs for each Month:
https://data.neonscience.org/api/v0/data/DP3.30010.001/GRSM/2015-08
https: